In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.disable_max_rows() # Disable 5_000 rows limit
data = pd.read_csv("../contests_code4rena.csv") # Set path accordingly

### Round prize pool value
Set this variable to round the prize pool values to the nearest multiple.

In [3]:
round_prize_pool = 1000

In [4]:
data["start"] = pd.to_datetime(data["start"])
data["end"] = pd.to_datetime(data["end"])
# Round prize_pool data to nearest multiple defined by 'round_prize_pool'
data["prize_pool"] = round(data["prize_pool"] / float(round_prize_pool)) * float(round_prize_pool)

In [5]:
data

,contest_report_repo,contest_sponsor,contest_desc,start,end,prize_pool,handle,prize_money,total_reports,high_all,high_solo,med_all,med_solo,gas_all
0,NaN,nibbl,NFT fractionalization protocol with guaranteed...,2022-06-21,2022-06-24,30000.0,Xxiaoming90,3657.41,4,0,0,3,1,0
1,NaN,nibbl,NFT fractionalization protocol with guaranteed...,2022-06-21,2022-06-24,30000.0,Hhansfriese,3019.73,4,0,0,2,1,1
2,NaN,nibbl,NFT fractionalization protocol with guaranteed...,2022-06-21,2022-06-24,30000.0,Rreassor,2424.37,3,0,0,1,1,1
3,NaN,nibbl,NFT fractionalization protocol with guaranteed...,2022-06-21,2022-06-24,30000.0,Uunforgiven,2368.15,2,0,0,1,1,0
4,NaN,nibbl,NFT fractionalization protocol with guaranteed...,2022-06-21,2022-06-24,30000.0,itsmeSTYJ,2339.34,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3621,NaN,slingshotfinance,Slingshot is a decentralized trading platform.,2021-02-16,2021-02-22,22000.0,nedodn,1902.35,2,0,0,1,1,1
3622,NaN,slingshotfinance,Slingshot is a decentralized trading platform.,2021-02-16,2021-02-22,22000.0,cmichel,1634.32,6,0,0,1,0,1
3623,NaN,slingshotfinance,Slingshot is a decentralized trading platform.,2021-02-16,2021-02-22,22000.0,onewayfunction,1570.29,6,0,0,0,0,4
3624,NaN,slingshotfinance,Slingshot is a decentralized trading platform.,2021-02-16,2021-02-22,22000.0,ncitron,1288.85,3,0,0,1,0,0


# Prize money distribution per prize pool (boxplot)

### Group prize pool data in multiple ranges
Set the variable below to the length of one interval for splitting the prize pool data. A larger number will group more contests together while a lower number will allow a more detailed visualization.

In [6]:
prize_pool_bin_size = 10000

In [7]:
plt_data = data.copy()
bins = range(0, int(plt_data["prize_pool"].max())+1, prize_pool_bin_size)
plt_data["bins"] = pd.cut(plt_data["prize_pool"], bins)
bins_labels = plt_data["bins"].cat.categories.astype('str').tolist()
plt_data = plt_data.sort_values("bins").reset_index(drop=True)
plt_data["bins"] = plt_data["bins"].astype('str')

In [8]:
box_plot = alt.Chart(plt_data).mark_boxplot(
    extent='min-max',
    size=45,
).encode(
    x=alt.X(
        'bins:N',
        sort=bins_labels,
        axis=alt.Axis(title='Prize pool ranges ($USD)', titleAngle=0, labelAngle=-45)
    ),
    y=alt.Y(
        'prize_money:Q',
        axis=alt.Axis(format='$,.0f', title='Prize money ($USD)')
    ),
    color=alt.Color('bins:O', legend=None, scale=alt.Scale(scheme='yelloworangered', domain=bins_labels))
)

In [9]:
max_line = box_plot.mark_line(
    strokeDash=[5]
).encode(
    y='max(prize_money):Q',
    color=alt.value('black')
)

In [10]:
bins_count_bar = box_plot.mark_bar().encode(
    y=alt.Y('distinct(contest_sponsor):Q', axis=alt.Axis(title='Number of contests', tickMinStep=1)),
    tooltip=['distinct(contest_sponsor):Q']
).properties(
    width=800,
    height=150
)

In [11]:
box_line = (box_plot + max_line).properties(
    title="Prize money distribution per prize pool",
    width=800,
    height=500
)

(box_line & bins_count_bar).configure_title(
    anchor='middle',
    fontSize=18,
    offset=15
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
)

alt.VConcatChart(...)

# Top warden's share per prize pool 

### Filter data by the top n<sup>th</sup> warden for each contest prize pool
Set this variable to plot the bar chart of the top n<sup>th</sup> warden's share for each contest prize pool. Run the notebook from here to the chart plot in order to refresh the chart.

In [12]:
filter_top_n_warden_per_contest = 1

In [13]:
plt_data = data.copy()
plt_data = plt_data[
    plt_data['prize_money'] == plt_data.groupby(['prize_pool'])['prize_money'].transform(
        lambda x: x.nlargest(filter_top_n_warden_per_contest).min()
    )
].reset_index(drop=True)
plt_data.drop_duplicates("prize_money", inplace=True) # Only keep one warden for each n_th position

In [14]:
bar_size = 25
grid_dash_size = 0

In [15]:
base = alt.Chart(plt_data).transform_calculate(
    prize_money_share="max(datum.prize_money) / datum.prize_pool"
).mark_bar(
    size=bar_size
).encode(
    x=alt.X(
        'prize_money_share:Q', 
        scale=alt.Scale(domain=[0, 1]), 
        axis=alt.Axis(format='%', title="Top warden's share", gridDash=[grid_dash_size], tickCount=10)
    ),
    y=alt.Y(
        'prize_pool:O',
        axis=alt.Axis(format='$,', title='Prize pool ($USD)', titleAngle=0, titleAnchor='start', offset=1)
    ),
    color=alt.Color(
        'handle:N', 
        scale=alt.Scale(scheme='category20b'), 
        legend=alt.Legend(
            title='Wardens', 
            orient='top', 
            padding=20, 
            labelFontSize=14, 
            titleFontSize=14, 
            columns=5,
            rowPadding=10,
            symbolType='square'
        )),
)

In [16]:
wardens_selector = alt.selection_multi(fields=['handle'], bind='legend')

In [17]:
bars = base.encode(
    opacity=alt.condition(wardens_selector, alt.value(1), alt.value(0.2))
).add_selection(wardens_selector)

In [18]:
rule_opacity = 0.6

### Magenta mean vertical line
Display the mean of the top n<sup>th</sup> warden's share across all contests 

In [19]:
mean_rule_line = alt.Chart(plt_data).transform_calculate(
    prize_money_share="max(datum.prize_money) / datum.prize_pool"
).mark_rule(
    color='mediumorchid',
    size=2,
    opacity=rule_opacity,
    strokeDash=[grid_dash_size]
).encode(
    x='mean(prize_money_share):Q'
)

In [20]:
mean_rule_label = mean_rule_line.mark_text(
    color='mediumorchid',
    opacity=rule_opacity,
    align='center',
    baseline='middle',
    fontSize=14,
    fontStyle='bold'
).encode(
    y=alt.value(-10),
    text=alt.Text('mean(prize_money_share):Q', format='.0%')
)

In [21]:
wardens_labels = base.mark_text(
    align='left',
    baseline='middle',
    dx=bar_size/5,
    fontSize=14,
    fontStyle='italic'
).encode(
    text='handle:N'
)

In [22]:
shares_labels = base.mark_text(
    align='right',
    baseline='middle',
    dx=-bar_size/5
).encode(
    text=alt.Text('prize_money:Q', format='$,.0f'),
    color=alt.value('white')
)

In [23]:
bar_spacing = 10

### Warden selection
Click on the legend to highlight the data for selected wardens – *use shift+left click for selecting multiple wardens*

In [24]:
(mean_rule_line + \
 mean_rule_label + \
 bars + \
 wardens_labels + \
 shares_labels
).properties(
    title="Top warden's share of prize pool per contest",
    width=870,
    height=alt.Step(bar_size + bar_spacing)
).configure_title(
    anchor='middle',
    fontSize=18,
    offset=15
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
)

alt.LayerChart(...)